In [1]:
import pandas as pd
import numpy as np
import pickle
import torch
import random
from torch_geometric.data import Data

In [ ]:
#load labels of OncoKB or OnGene
datasetType = "oncokb"
if datasetType == "oncokb":
    ongene = pd.read_table('../data/OncoKB_cancerGeneList.tsv', index_col=None, header =0, sep="\t")
    ongene_name = list(ongene.iloc[:, 0].values)
    
if datasetType == "ongene":
    ongene = pd.read_table('../data/ongene_human.txt', index_col=None, header =0, sep="\t")
    ongene_name = list(ongene.iloc[:, 1].values)

print(len(ongene_name))

#selecet positive samples of OncoKB or OnGene
gene_features = pd.read_table('../data/pan/string_850/mut_features_miRNA_sub_du_2.0.txt', index_col=0, header =0, sep="\t")
gene_list = list(gene_features.index.values)
all_on = []
for gene in ongene_name:
    if gene in gene_list:
        all_on.append(gene)
print(len(all_on))

In [ ]:
#selecet the original positive samples
driver_list = pd.read_table(filepath_or_buffer='../data/796true.txt', header=None, index_col=None, names=['name'], dtype='str')
driver_gene = list(driver_list['name'].values)
all_driver = []
for gene in driver_gene:
    if gene in gene_list:
        all_driver.append(gene)
print(len(all_driver))

In [ ]:
#selecet positive samples of test set
test_list = []
for gene in all_on:
    if gene not in all_driver:
        test_list.append(gene)
print(len(test_list))
test_gene = pd.DataFrame(test_list)
test_gene.to_csv(path_or_buf='../data/pan/string_850/%s_test_2.0.txt' %datasetType, sep='\t', header=False,index=False)

In [5]:
cenerate label index for each gene
df1 = pd.read_table('../data/pan/string_850/mut_features_miRNA_sub_du_2.0.txt', index_col=0, header =0, sep="\t")
gene_index = list(df1.index.values)
df2 = pd.read_table('../data/796true.txt', header=None, index_col=None, names=['name'], dtype='str')
df3 = pd.read_table('../data/2187false.txt', header=None, index_col=None, names=['name'], dtype='str')
df4 = pd.read_table('../data/pan/string_850/%s_test_2.0.txt' %datasetType, header=None, index_col=None, names=['name'], dtype='str')
konwn_T_list = list(df2['name'].values)
konwn_F_list = list(df3['name'].values)
independ_list = list(df4['name'].values)

driver_index = []
normal_index = []
unkown_index = []
independ_index = []
for index,gene in enumerate(gene_index):
    flag = 0
    if gene in independ_list:
        independ_index.append(index)
        flag = 1
    if flag==0:
        if gene in konwn_T_list:
            driver_index.append(index)
            flag = 1
        if gene in konwn_F_list:
            normal_index.append(index)
            flag = 1
    if flag==0:
        unkown_index.append(index)

In [6]:
#divide the positive samples of train set and independ test set
def generate_bool_list(lst):
    return [bool(x) for x in lst]

y_list1 = [0]*len(gene_index)
for index,gene in enumerate(gene_index):
    if gene in konwn_T_list:
        y_list1[index] = 1
    if gene in independ_list:
        y_list1[index] = 1
        
y_te1 = generate_bool_list(y_list1)
y1 = torch.tensor(np.array(y_list1).reshape(len(gene_index),-1))
y_t1 = torch.tensor(np.array(y_te1).reshape(len(gene_index),-1))
y_data1 =Data(y=y1,y_te=y_t1)
if datasetType == "oncokb":
    torch.save(y_data1, '../data/pan/string_850/Y_independ_train_data.pkl')
    
if datasetType == "ongene":
    torch.save(y_data1, '../data/pan/string_850/Y_ongene_independ_train_data.pkl')

y_list2 = [0]*len(gene_index)
for index,gene in enumerate(gene_index):
    if gene in independ_list:
        y_list2[index] = 1
        
y_te2 = generate_bool_list(y_list2)
y2 = torch.tensor(np.array(y_list2).reshape(len(gene_index),-1))
y_t2 = torch.tensor(np.array(y_te2).reshape(len(gene_index),-1))
y_data2 =Data(y=y2,y_te=y_t2)

if datasetType == "oncokb":
    torch.save(y_data2, '../data/pan/string_850/Y_independ_test_data.pkl')
    
if datasetType == "ongene":
    torch.save(y_data2, '../data/pan/string_850/Y_ongene_independ_test_data.pkl')

In [7]:
#create dataset for ten five-fold cross-validations
def create_k_sets():
    data = pd.read_table("../data/pan/string_850/mut_features_miRNA_sub_du_2.0.txt", index_col=0, header =0, sep="\t")
    length = len(data.values)
    index_array = np.arange(0,length,1)
    
    tr_list = [False]*length
    for suoyin1 in driver_index:
        tr_list[suoyin1] = True
    for suoyin2 in normal_index:
        tr_list[suoyin2] = True
        
    te_list = [False]*length
    for suoyin4 in unkown_index:
        te_list[suoyin4] = True
    for suoyin5 in independ_index:
        te_list[suoyin5] = True
            
    inputs = []
    inputs.append(tr_list)
    inputs.append(te_list)
    k_sets = np.array(inputs)
    print(k_sets.shape)

    if datasetType == "oncokb":
        with open('../data/pan/string_850/k_sets_independ.pkl', 'wb') as f:
            pickle.dump(k_sets, f)
            
    if datasetType == "ongene":
        with open('../data/pan/string_850/k_sets_ongene_independ.pkl', 'wb') as f:
            pickle.dump(k_sets, f)
            
create_k_sets()